In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/creditcardfraud


In [5]:
!pip install xgboost imbalanced-learn plotly streamlit shap -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd


dataset_path = '/kaggle/input/creditcardfraud/creditcard.csv'


df = pd.read_csv(dataset_path)


print(df.head())


   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28 

In [ ]:

for col in df.columns:
    if df[col].dtype == 'object':
        df[col].fillna('Unknown', inplace=True)
    else:
        df[col].fillna(df[col].median(), inplace=True)


/tmp/ipython-input-2757864259.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [ ]:

import numpy as np


df['Amount_log'] = np.log1p(df['Amount'])


df['Amount_decimal'] = df['Amount'] - df['Amount'].astype(int)
df['Amount_isRound'] = (df['Amount'] == df['Amount'].astype(int)).astype(int)


print(df[['Amount', 'Amount_log', 'Amount_decimal', 'Amount_isRound']].head())


   Amount  Amount_log  Amount_decimal  Amount_isRound
0  149.62    5.014760            0.62               0
1    2.69    1.305626            0.69               0
2  378.66    5.939276            0.66               0
3  123.50    4.824306            0.50               0
4   69.99    4.262539            0.99               0


In [28]:
le = LabelEncoder()
cat_cols = df.select_dtypes(include=['object']).columns

for col in cat_cols:
    df[col + '_enc'] = le.fit_transform(df[col].astype(str))
    df.drop(columns=[col], inplace=True)

print('Shape after encoding:', df.shape)


Shape after encoding: (284807, 34)


In [ ]:


from sklearn.model_selection import train_test_split  


X = df.drop(['Class', 'Time'], axis=1, errors='ignore')
y = df['Class'] 


from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print("Train shape:", X_train.shape)
print("Test  shape:", X_test.shape)


Train shape: (227845, 32)
Test  shape: (56962, 32)


In [ ]:


from imblearn.over_sampling import SMOTE  

smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)


print("Original fraud ratio:", y_train.mean())
print("Post-SMOTE fraud ratio:", y_train_sm.mean())
print("SMOTE applied: X_train_sm shape:", X_train_sm.shape)


Original fraud ratio: 0.001729245759178389
Post-SMOTE fraud ratio: 0.5
SMOTE applied: X_train_sm shape: (454902, 32)


In [ ]:


import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# 1) XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_model.fit(X_train_sm, y_train_sm)

# 2) Random Forest
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train_sm, y_train_sm)

# 3) Logistic Regression
lr_model = LogisticRegression(
    max_iter=1000,
    random_state=42
)
lr_model.fit(X_train_sm, y_train_sm)

print("Models trained successfully.")


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [06:34:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Models trained successfully.


In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [ ]:

xgb_pred = xgb_model.predict(X_test)
rf_pred  = rf_model.predict(X_test)
lr_pred  = lr_model.predict(X_test)


ensemble_pred = (
    0.4 * xgb_pred +
    0.3 * rf_pred +
    0.3 * lr_pred
).round().astype(int)


print('Accuracy:', accuracy_score(y_test, ensemble_pred))
print('Precision:', precision_score(y_test, ensemble_pred))
print('Recall:', recall_score(y_test, ensemble_pred))
print('F1-score:', f1_score(y_test, ensemble_pred))
print('AUC-ROC:', roc_auc_score(y_test, ensemble_pred))


Accuracy: 0.9979108879603946
Precision: 0.4461538461538462
Recall: 0.8877551020408163
F1-score: 0.5938566552901023
AUC-ROC: 0.942927916805439


In [ ]:


import pickle  


model_package = {
    'scaler': scaler,
    'xgb': xgb_model,
    'rf': rf_model,
    'lr': lr_model
}


with open('fraud_detection_model.pkl', 'wb') as f:
    pickle.dump(model_package, f)

print("Model saved to fraud_detection_model.pkl")


Model saved to fraud_detection_model.pkl


2nd Dataset

In [ ]:
import pandas as pd


df_tr = pd.read_csv('train_transaction.csv')
df_id = pd.read_csv('train_identity.csv')

print("Transaction shape:", df_tr.shape)
print("Identity shape:", df_id.shape)


Transaction shape: (96310, 394)
Identity shape: (144233, 41)


In [ ]:
df_ieee = df_tr.merge(df_id, on='TransactionID', how='left')

print("Merged shape:", df_ieee.shape)
df_ieee.head()


Merged shape: (96310, 434)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [4]:
for col in df_ieee.columns:
    if df_ieee[col].dtype == 'object':
        df_ieee[col].fillna('Unknown', inplace=True)
    else:
        df_ieee[col].fillna(df_ieee[col].median(), inplace=True)


/tmp/ipython-input-665617926.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ieee[col].fillna(df_ieee[col].median(), inplace=True)
/tmp/ipython-input-665617926.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [ ]:
import numpy as np
df_ieee['TransactionAmt_log'] = np.log1p(df_ieee['TransactionAmt'])


df_ieee['TransactionAmt_decimal'] = df_ieee['TransactionAmt'] - df_ieee['TransactionAmt'].astype(int)
df_ieee['TransactionAmt_isRound']  = (df_ieee['TransactionAmt'] == df_ieee['TransactionAmt'].astype(int)).astype(int)


In [6]:
df_ieee['Transaction_hour'] = (df_ieee['TransactionDT'] // 3600) % 24
df_ieee['Transaction_day'] = (df_ieee['TransactionDT'] // (3600*24)) % 7


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_cols = df_ieee.select_dtypes(include=['object']).columns
for col in cat_cols:
    df_ieee[col + '_enc'] = le.fit_transform(df_ieee[col].astype(str))
    df_ieee.drop(columns=[col], inplace=True)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_ieee = df_ieee.drop(['isFraud', 'TransactionID'], axis=1, errors='ignore')
y_ieee = df_ieee['isFraud']

print(f"IEEE Features shape: {X_ieee.shape}")
print(f"IEEE Fraud rate: {y_ieee.mean():.4f}")


scaler_ieee = StandardScaler()
X_ieee_scaled = scaler_ieee.fit_transform(X_ieee)

X_ieee_train, X_ieee_test, y_ieee_train, y_ieee_test = train_test_split(
    X_ieee_scaled, y_ieee,
    test_size=0.2,
    stratify=y_ieee,
    random_state=42
)

print(f"Train shape: {X_ieee_train.shape}")
print(f"Test shape: {X_ieee_test.shape}")


IEEE Features shape: (96310, 437)
IEEE Fraud rate: 0.0259
Train shape: (77048, 437)
Test shape: (19262, 437)


In [9]:
from imblearn.over_sampling import SMOTE

smote_ieee = SMOTE(random_state=42)
X_ieee_train_sm, y_ieee_train_sm = smote_ieee.fit_resample(X_ieee_train, y_ieee_train)

print(f"Original fraud rate: {y_ieee_train.mean():.4f}")
print(f"Post-SMOTE fraud rate: {y_ieee_train_sm.mean():.4f}")
print(f"SMOTE training shape: {X_ieee_train_sm.shape}")


Original fraud rate: 0.0259
Post-SMOTE fraud rate: 0.5000
SMOTE training shape: (150112, 437)


In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

print("Training IEEE-CIS models...")


xgb_ieee = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_ieee.fit(X_ieee_train_sm, y_ieee_train_sm)
print("XGBoost IEEE trained")


rf_ieee = RandomForestClassifier(
    n_estimators=150,
    max_depth=10,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)
rf_ieee.fit(X_ieee_train_sm, y_ieee_train_sm)
print(" Random Forest IEEE trained")


lr_ieee = LogisticRegression(
    max_iter=1000,
    random_state=42,
    C=1.0
)
lr_ieee.fit(X_ieee_train_sm, y_ieee_train_sm)
print("Logistic Regression IEEE trained")


Training IEEE-CIS models...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [05:32:17] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ XGBoost IEEE trained
✅ Random Forest IEEE trained
✅ Logistic Regression IEEE trained


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


xgb_pred_ieee = xgb_ieee.predict(X_ieee_test)
rf_pred_ieee = rf_ieee.predict(X_ieee_test)
lr_pred_ieee = lr_ieee.predict(X_ieee_test)


ensemble_ieee_pred = (
    0.4 * xgb_pred_ieee +
    0.3 * rf_pred_ieee +
    0.3 * lr_pred_ieee
).round().astype(int)


print("🎯 IEEE-CIS MODEL RESULTS:")
print(f"Accuracy:  {accuracy_score(y_ieee_test, ensemble_ieee_pred):.4f}")
print(f"Precision: {precision_score(y_ieee_test, ensemble_ieee_pred):.4f}")
print(f"Recall:    {recall_score(y_ieee_test, ensemble_ieee_pred):.4f}")
print(f"F1-Score:  {f1_score(y_ieee_test, ensemble_ieee_pred):.4f}")
print(f"AUC-ROC:   {roc_auc_score(y_ieee_test, ensemble_ieee_pred):.4f}")


🎯 IEEE-CIS MODEL RESULTS:
Accuracy:  0.9635
Precision: 0.3742
Recall:    0.6124
F1-Score:  0.4646
AUC-ROC:   0.7926


In [ ]:
import pickle


ieee_model_package = {
    'scaler': scaler_ieee,
    'xgb': xgb_ieee,
    'rf': rf_ieee,
    'lr': lr_ieee,
    'feature_names': X_ieee.columns.tolist()
}


with open('ieee_fraud_model.pkl', 'wb') as f:
    pickle.dump(ieee_model_package, f)

print(" IEEE-CIS model saved as 'ieee_fraud_model.pkl'")
print(" Ready for ensemble with ULB model!")


✅ IEEE-CIS model saved as 'ieee_fraud_model.pkl'
📊 Ready for ensemble with ULB model!


3rd Dataset

In [ ]:
import kagglehub


sparkov_path = kagglehub.dataset_download("kartik2112/fraud-detection")
print("Sparkov dataset path:", sparkov_path)


100%|██████████| 202M/202M [00:01<00:00, 139MB/s]

Extracting files...


Sparkov dataset path: /root/.cache/kagglehub/datasets/kartik2112/fraud-detection/versions/1


In [ ]:
import pandas as pd
import os


df_sparkov = pd.read_csv(os.path.join(sparkov_path, "fraudTrain.csv"))

print(f"Sparkov dataset shape: {df_sparkov.shape}")
print(f"Fraud rate: {df_sparkov['is_fraud'].mean():.4f}")
print(f"Columns: {list(df_sparkov.columns)}")
df_sparkov.head()


Sparkov dataset shape: (1296675, 23)
Fraud rate: 0.0058
Columns: ['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


In [ ]:

for col in df_sparkov.columns:
    if df_sparkov[col].dtype == 'object':
        df_sparkov[col].fillna('Unknown', inplace=True)
    else:
        df_sparkov[col].fillna(df_sparkov[col].median(), inplace=True)


import numpy as np
df_sparkov['amt_log'] = np.log1p(df_sparkov['amt'])
df_sparkov['amt_decimal'] = df_sparkov['amt'] - df_sparkov['amt'].astype(int)
df_sparkov['amt_isRound'] = (df_sparkov['amt'] == df_sparkov['amt'].astype(int)).astype(int)


df_sparkov['trans_date_trans_time'] = pd.to_datetime(df_sparkov['trans_date_trans_time'])
df_sparkov['hour'] = df_sparkov['trans_date_trans_time'].dt.hour
df_sparkov['day'] = df_sparkov['trans_date_trans_time'].dt.day
df_sparkov['month'] = df_sparkov['trans_date_trans_time'].dt.month


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_cols = ['merchant', 'category', 'first', 'last', 'gender', 'street', 'city', 'state', 'job']
for col in cat_cols:
    if col in df_sparkov.columns:
        df_sparkov[col + '_enc'] = le.fit_transform(df_sparkov[col].astype(str))
        df_sparkov.drop(columns=[col], inplace=True)

print(f"After preprocessing: {df_sparkov.shape}")


/tmp/ipython-input-3939043663.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_sparkov[col].fillna(df_sparkov[col].median(), inplace=True)
/tmp/ipython-input-3939043663.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

After preprocessing: (1296675, 29)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


non_numeric = X_sparkov.select_dtypes(include=['object']).columns.tolist()
print("Non-numeric columns to drop:", non_numeric)


X_sparkov = X_sparkov.drop(columns=non_numeric)


scaler_sparkov = StandardScaler()
X_sparkov_scaled = scaler_sparkov.fit_transform(X_sparkov)


X_sparkov_train, X_sparkov_test, y_sparkov_train, y_sparkov_test = train_test_split(
    X_sparkov_scaled,
    y_sparkov,
    test_size=0.2,
    stratify=y_sparkov,
    random_state=42
)

print("Scaled and split shapes:", X_sparkov_train.shape, X_sparkov_test.shape)


Non-numeric columns to drop: ['dob']
Scaled and split shapes: (1037340, 25) (259335, 25)


In [18]:
from imblearn.over_sampling import SMOTE

smote_sparkov = SMOTE(random_state=42)
X_sparkov_train_sm, y_sparkov_train_sm = smote_sparkov.fit_resample(X_sparkov_train, y_sparkov_train)

print(f"Post-SMOTE fraud rate: {y_sparkov_train_sm.mean():.4f}")


Post-SMOTE fraud rate: 0.5000


In [ ]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

print("Training Sparkov models...")


xgb_sparkov = xgb.XGBClassifier(
    n_estimators=200, max_depth=6, learning_rate=0.1,
    subsample=0.8, colsample_bytree=0.8,
    random_state=42, use_label_encoder=False, eval_metric='logloss'
)
xgb_sparkov.fit(X_sparkov_train_sm, y_sparkov_train_sm)


rf_sparkov = RandomForestClassifier(
    n_estimators=150, max_depth=10,
    random_state=42, n_jobs=-1
)
rf_sparkov.fit(X_sparkov_train_sm, y_sparkov_train_sm)


lr_sparkov = LogisticRegression(max_iter=1000, random_state=42)
lr_sparkov.fit(X_sparkov_train_sm, y_sparkov_train_sm)

print("All Sparkov models trained successfully!")


Training Sparkov models...


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [05:54:49] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


✅ All Sparkov models trained successfully!


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


xgb_pred_sparkov = xgb_sparkov.predict(X_sparkov_test)
rf_pred_sparkov = rf_sparkov.predict(X_sparkov_test)
lr_pred_sparkov = lr_sparkov.predict(X_sparkov_test)


ensemble_sparkov_pred = (
    0.4 * xgb_pred_sparkov +
    0.3 * rf_pred_sparkov +
    0.3 * lr_pred_sparkov
).round().astype(int)

print("🎯 SPARKOV MODEL RESULTS:")
print(f"Accuracy:  {accuracy_score(y_sparkov_test, ensemble_sparkov_pred):.4f}")
print(f"Precision: {precision_score(y_sparkov_test, ensemble_sparkov_pred):.4f}")
print(f"Recall:    {recall_score(y_sparkov_test, ensemble_sparkov_pred):.4f}")
print(f"F1-Score:  {f1_score(y_sparkov_test, ensemble_sparkov_pred):.4f}")
print(f"AUC-ROC:   {roc_auc_score(y_sparkov_test, ensemble_sparkov_pred):.4f}")


🎯 SPARKOV MODEL RESULTS:
Accuracy:  0.9840
Precision: 0.2491
Recall:    0.8801
F1-Score:  0.3883
AUC-ROC:   0.9323


In [ ]:
import pickle


sparkov_model_package = {
    'scaler': scaler_sparkov,
    'xgb': xgb_sparkov,
    'rf': rf_sparkov,
    'lr': lr_sparkov,
    'feature_names': X_sparkov.columns.tolist()
}

# Save
with open('sparkov_fraud_model.pkl', 'wb') as f:
    pickle.dump(sparkov_model_package, f)

print("Sparkov model saved as 'sparkov_fraud_model.pkl'")


✅ Sparkov model saved as 'sparkov_fraud_model.pkl'


In [36]:
import pickle

with open('/content/fraud_detection_model.pkl', 'rb') as f:
    ulb_pkg = pickle.load(f)

with open('/content/ieee_fraud_model.pkl', 'rb') as f:
    ieee_pkg = pickle.load(f)

with open('/content/sparkov_fraud_model.pkl', 'rb') as f:
    sparkov_pkg = pickle.load(f)

print("All model packages loaded correctly")


All model packages loaded correctly
